In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import math #The Python math module
#import iexfinance
from scipy import stats #The SciPy stats module

In [4]:
### Start of Analysis Programm

# Analysis steps 
# (a) get actual S&P500 data with relevant KPI
locpath1 = "C:/Users/marcw/01_projects/jupyterlab/03_algorithmic_trading/"
ana_df = pd.read_excel(locpath1+"fin_df.xlsx")

# (b) get industry category for each S&P500 company

spind_df = pd.read_csv(locpath1+"S&P500-Info.csv")

spind_df = spind_df.loc[:,[ 'Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]            

# (c) merge both data sets together

ana_df = pd.merge(left=ana_df, right=spind_df, how='left', left_on='Ticker', right_on='Symbol')
            

# (d) calculate percentile variables for each KPI

ana_df['spread_52LowHigh'] = ana_df['Adjusted 52 week high'] - ana_df['Adjusted 52 week low']
ana_df['spreadrel_52LowHigh'] = ana_df['spread_52LowHigh'] / ana_df['Adjusted 52 week low']

ana_df['spread_52LowLatest'] = ana_df['Price'] - ana_df['Adjusted 52 week low']
ana_df['spreadrel_52LowLatest'] = ana_df['spread_52LowLatest'] / ana_df['Adjusted 52 week low']

ana_df['spreadrel_52LowHigh_pc'] = pd.Series(['N/A'] * len(ana_df))
ana_df['spreadrel_52LowLatest_pc'] = pd.Series(['N/A'] * len(ana_df))
ana_df['Price to Earnings Ratio_pc'] = pd.Series(['N/A'] * len(ana_df))
ana_df['One-Year Price Return_pc'] = pd.Series(['N/A'] * len(ana_df))
ana_df['Six-Month Price Return_pc'] = pd.Series(['N/A'] * len(ana_df))
ana_df['Three-Month Price Return_pc'] = pd.Series(['N/A'] * len(ana_df))
ana_df['One-Month Price Return_pc'] = pd.Series(['N/A'] * len(ana_df))


pc_attributes = [
                'spreadrel_52LowHigh', 
                'spreadrel_52LowLatest',
                'Price to Earnings Ratio',
                'One-Year Price Return',
                'Six-Month Price Return',
                'Three-Month Price Return',
                'One-Month Price Return'
                ]

for row in ana_df.index:
    for pc_attribute in pc_attributes:
        ana_df.loc[row, f'{pc_attribute}_pc'] = stats.percentileofscore(ana_df[f'{pc_attribute}'],
                                                                        ana_df.loc[row, f'{pc_attribute}'])/100
#ana_df

#ana_df.to_excel(locpath1+"ana_df.xlsx", sheet_name='Tabelle1')   

# (e) calculate combined kpi

ana_df['Twelve months Earnings Positiv Ind'] = np.where(ana_df['Twelve months Earnings per Share'] < 0, 0, 1)

ana_df['kpi_value_1'] = 1 - ana_df['Price to Earnings Ratio_pc']
ana_df['kpi_momentum_1'] = (ana_df['One-Year Price Return_pc'] + ana_df['Six-Month Price Return_pc'] +
                            ana_df['Three-Month Price Return'] + ana_df['One-Month Price Return'])/4
ana_df['kpi_momentum_2'] = ana_df['spreadrel_52LowLatest_pc']

ana_df['kpi_1'] = (ana_df['kpi_value_1'] + ana_df['kpi_momentum_1'])/2
ana_df['kpi_2'] = ((1 - ana_df['Price to Earnings Ratio_pc']) + ana_df['spreadrel_52LowLatest_pc'])/2

#Store raw data extract in to xls.
locpath1 = "C:/Users/marcw/01_projects/jupyterlab/03_algorithmic_trading/"
ana_df.to_excel(locpath1+"ana_df.xlsx", sheet_name='Tabelle1')   

ana_df